In [1]:
import numpy as np
from glob import glob
import pandas as pd

files = glob('data/*.csv')
totals = ['Total_PorosityQuantity', 'Total_PorosityQuality', 'Total_UnfilledZones', 'Total_FillingQuality', 'TOTAL_QUALITY']

df_list = list()
target = list()

for i, file in enumerate(files[:100]):
    df = pd.read_csv(file)
    target.append(df.TOTAL_QUALITY.unique()[0])
    df = df.drop(axis=1, columns=totals)  
    df['id'] = i
    
    ts_cols = [col for col in df if col.endswith(('Time', 'id', 'Flow rate', 'Pressure'))]
    df_list.append(df[ts_cols])

df = pd.concat(df_list)
df.head()

,Time,Flow rate,Zone2_Pressure,Zone3_Pressure,Zone4_Pressure,Zone5_Pressure,Zone6_Pressure,Zone7_Pressure,Zone9_Pressure,Zone10_Pressure,...,Zone22_Pressure,Zone23_Pressure,Zone24_Pressure,Zone26_Pressure,Zone27_Pressure,Zone28_Pressure,Zone29_Pressure,Zone30_Pressure,Zone31_Pressure,id
0,0.065529,6.727330e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.094341,7.170040e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.116504,7.614100e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.165700,8.404510e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.186822,8.817000e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [2]:
df.id.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
      dtype=int64)

In [3]:
type(df.id.values)

numpy.ndarray

In [4]:
from tsfresh import extract_features
from tsfresh.utilities.distribution import MultiprocessingDistributor
distributor = MultiprocessingDistributor(n_workers=8, disable_progressbar=False, progressbar_title="Feature Extraction")
extracted_features = extract_features(df, column_id='id', column_sort='Time', distributor=distributor)
extracted_features.to_csv('temp/extracted_features.csv')

Feature Extraction: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [45:39<00:00, 68.50s/it]


### READ EXTRACTED FEATURES FROM CSV

In [7]:
import pandas as pd

extracted_features = pd.read_csv('temp/extracted_features.csv')
extracted_features.head()

,id,Flow rate__abs_energy,Flow rate__absolute_sum_of_changes,"Flow rate__agg_autocorrelation__f_agg_""mean""__maxlag_40","Flow rate__agg_autocorrelation__f_agg_""median""__maxlag_40","Flow rate__agg_autocorrelation__f_agg_""var""__maxlag_40","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""",...,Zone9_Pressure__symmetry_looking__r_0.9,Zone9_Pressure__symmetry_looking__r_0.9500000000000001,Zone9_Pressure__time_reversal_asymmetry_statistic__lag_1,Zone9_Pressure__time_reversal_asymmetry_statistic__lag_2,Zone9_Pressure__time_reversal_asymmetry_statistic__lag_3,Zone9_Pressure__value_count__value_-1,Zone9_Pressure__value_count__value_0,Zone9_Pressure__value_count__value_1,Zone9_Pressure__variance,Zone9_Pressure__variance_larger_than_standard_deviation
0,0,1.031236e-07,0.000062,0.0,0.0,0.0,0.000012,-0.243467,-4.506888e-08,1.958974e-08,...,1.0,1.0,3.410462e+11,6.741525e+11,1.002176e+12,0.0,691.0,0.0,3.671472e+08,1.0
1,1,1.042159e-07,0.000064,0.0,0.0,0.0,0.000012,-0.253883,-4.662547e-08,1.926688e-08,...,1.0,1.0,4.947107e+11,9.807605e+11,1.460443e+12,0.0,705.0,0.0,4.612232e+08,1.0
2,2,9.353593e-08,0.000079,0.0,0.0,0.0,0.000013,-0.410989,-7.622757e-08,1.802454e-08,...,1.0,1.0,8.272206e+10,1.634417e+11,2.445991e+11,0.0,700.0,0.0,3.912031e+08,1.0
3,3,1.027388e-07,0.000061,0.0,0.0,0.0,0.000013,-0.381023,-7.325205e-08,1.884132e-08,...,1.0,1.0,4.910345e+11,9.765174e+11,1.455599e+12,0.0,692.0,0.0,4.928808e+08,1.0
4,4,1.049617e-07,0.000068,0.0,0.0,0.0,0.000013,-0.428986,-7.536972e-08,1.654617e-08,...,1.0,1.0,3.846651e+10,7.558987e+10,1.115603e+11,0.0,747.0,0.0,1.040071e+08,1.0


### GENERATE FIRST MODEL AND TEST

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

extracted_features = extracted_features.dropna(axis=1)
extracted_features = extracted_features.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

X_train, X_test, y_train, y_test = train_test_split(extracted_features, target, test_size=.3)
rf_model = RandomForestClassifier(n_estimators=100).fit(X_train, np.ravel(y_train))
y_pred = rf_model.predict(X_test)

accuracy_score(y_test, y_pred)

0.7